In [40]:
import os
import time
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
from oandapyV20 import API
from oandapyV20.endpoints import instruments, orders
import warnings
from dotenv import load_dotenv
import traceback
import threading
np.NaN = np.nan
import pandas_ta as ta
# -----------------------------
# 0️⃣ Setup
# -----------------------------
load_dotenv()
account_id = os.getenv('OANDA_ACCOUNT_ID_HEDGE')
access_key = os.getenv('OANDA_ACCESS_KEY')
api = API(access_token=access_key)
warnings.filterwarnings("ignore")


# -----------------------------
# 1️⃣ Helper functions
# -----------------------------
def get_candles(symbol: str, count: int = 20, granularity: str = 'M1'):
    params = {"count": count, "granularity": granularity, "price": "MBA"}
    r = instruments.InstrumentsCandles(instrument=symbol, params=params)
    response = api.request(r)
    return response['candles']


def candles_to_df(candles):
    records = []
    for c in candles:
        records.append({
            "time": pd.to_datetime(c["time"]),
            "complete": c["complete"],
            "volume": c["volume"],
            "mid_o": float(c["mid"]["o"]),
            "mid_h": float(c["mid"]["h"]),
            "mid_l": float(c["mid"]["l"]),
            "mid_c": float(c["mid"]["c"]),
            "bid_o": float(c["bid"]["o"]),
            "bid_h": float(c["bid"]["h"]),
            "bid_l": float(c["bid"]["l"]),
            "bid_c": float(c["bid"]["c"]),
            "ask_o": float(c["ask"]["o"]),
            "ask_h": float(c["ask"]["h"]),
            "ask_l": float(c["ask"]["l"]),
            "ask_c": float(c["ask"]["c"]),
        })
    return pd.DataFrame(records)


def format_price(price, instrument):
    # JPY pairs → 3 decimals, others → 5 decimals
    if "JPY" in instrument:
        return str(round(price, 3))
    else:
        return str(round(price, 5))


def place_order(units: int, side: str, sl_price: float, tp_price: float, symbol: str):
    data = {
        "order": {
            "instrument": symbol,
            "units": str(units if side == "buy" else -units),
            "type": "MARKET",
            "positionFill": "DEFAULT",
            "stopLossOnFill": {"price": format_price(sl_price, symbol)},
            "takeProfitOnFill": {"price": format_price(tp_price, symbol)}
        }
    }
    print(format_price(sl_price, symbol), "Stop loss")
    print(format_price(tp_price, symbol),"Take profit")
    r = orders.OrderCreate(accountID=account_id, data=data)
    response = api.request(r)
    print(f"[{datetime.now(timezone.utc)}] Order placed: {response}")



In [41]:
import os

In [42]:
os.chdir('..')

In [43]:
from strategies.vwap_rsi_scalping import strategy  # Your custom strategy function


In [ ]:
data = get_candles("EUR_JPY", count=500, granularity="M1")
df = candles_to_df(data)
df = df[df['complete']]
df.iloc[-1]['ask_o'] - df.iloc[-1]['bid_o']

np.float64(0.00034000000000000696)

In [45]:
df['Open'], df['High'], df['Low'], df['Close'], df['Volume'] = \
                df['mid_o'], df['mid_h'], df['mid_l'], df['mid_c'], df['volume']
df = df.sort_values('time')
df.set_index('time', inplace=True)
df = strategy(df,15, 1.5)        

In [46]:
df.columns

Index(['time', 'complete', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
       'bid_o', 'bid_h', 'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c',
       'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'RSI', 'BBL_14_2.0',
       'BBM_14_2.0', 'BBU_14_2.0', 'BBB_14_2.0', 'BBP_14_2.0', 'atr',
       'upper_band', 'lower_band', 'VWAPSignal', 'TotalSignal'],
      dtype='object')

In [47]:
last = df.iloc[-1]
signal = last['TotalSignal']
atr = last['atr']


In [48]:
last.atr

np.float64(0.00015591012170106293)

In [49]:
sl_distance = last.atr * 5
tp_distance = last.atr * 5

In [50]:
sl_distance , tp_distance

(np.float64(0.0007795506085053147), np.float64(0.0007795506085053147))

In [51]:
last.Close - sl_distance , last.Close, last.Close + tp_distance

(np.float64(1.7929904493914948),
 np.float64(1.79377),
 np.float64(1.7945495506085054))

In [ ]:
place_order(symbol="EUR_JPY", side="buy", sl_price=last.Close - sl_distance,tp_price= last.Close + tp_distance, units=1000)

1.79299 Stop loss
1.79455 Take profit
[2025-08-26 16:13:57.843714+00:00] Order placed: {'orderCreateTransaction': {'id': '38848', 'accountID': '101-011-24509333-004', 'userID': 24509333, 'batchID': '38848', 'requestID': '97438133407906240', 'time': '2025-08-26T16:13:57.740980502Z', 'type': 'MARKET_ORDER', 'instrument': 'EUR_AUD', 'units': '1000', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '1.79455', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '1.79299', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderFillTransaction': {'id': '38849', 'accountID': '101-011-24509333-004', 'userID': 24509333, 'batchID': '38848', 'requestID': '97438133407906240', 'time': '2025-08-26T16:13:57.740980502Z', 'type': 'ORDER_FILL', 'orderID': '38848', 'instrument': 'EUR_AUD', 'units': '1000', 'requestedUnits': '1000', 'price': '1.79406', 'pl': '0.0000', 'quotePL': '0', 'financing': '0.0000', 'baseFinancing': '0', 'commission': '0.00